# Lesson 5b: Recurrent Neural Networks: Categorization

Recurrent Neural Networks (RNNs) can be used in many different ways, such as classification, single-step prediction, and the generation of an entire sequence. 

* **Classification**: the input is a sequence, and the output is a single category - this is the focus of this assignment. (Alternatively, a sequence of categories could be generated, one for each partial sequence as it is processed).

* **Prediction**: the input is a sequence, and the output is a prediction for the next element in the sequence. You will explore this in lesson 5b.

* **Sequence Generation** (Seq-to-Seq): both the input and the output are entire sequences. For example, RNN-based language translation may take in an input sequence (of characters or word tokens) in English, and generate as output a sequence (of characters or word tokens) in French.

RNNs can be used to process inputs that occur naturally in time (such as an audio recording of speech or music represented as a stream of timestamped MIDI messages), but they can also be applied to material that has an order to it, even if it's not necessarly temporal in natures, such as written text (which can be read one character or one word at a time) or even written numbers or math equations (which can be read one digit or symbol at a time, from left to right, for instance.)  This is the problem we investigate today: looking at numbers such as "1423" as a sequence of digits ['1', '4', '2', '3'].

Our problem comes curtosy of Distinguished Professor Douglas R. Hofstadter of Indiana University, author of books such as _Gödel, Escher, Bach: an Eternal Golden Braid_. Hofstadter writes [private communication, shared with permission)]:

---

_Lately, I have been musing about the seeming power of deep neural nets.  They learn to recognize members of all sorts of categories, when those members (and non-members) are fed to them as patterns of symbols or of pixels.  So, how about the following challenges involving the natural numbers?_

* To recognize the even numbers, expressed in base 3.
     (Specifically, 0, 2, 11, 20, 22, 101, 110, 112,...)
* To recognize the multiples of 3, expressed in base 10.
* To recognize the multiples of 9, expressed in base 10.
* To recognize the multiples of 7, expressed in base 10.
* To recognize the multiples of 29, expressed in base 10.

_(I suppose that if a net can learn any particular one of the above list, it can learn all of them.  Just a guess...)_

 _Moving right along, how about the following somewhat harder challenges?_

* To recognize the correct integer additions, expressed either in base 2 or in base 10.  (For example, the string “12+29=41”.)
* To recognize the correct integer multiplications, expressed either in base 2 or in base 10.  (For example, the string “12x29=348”.)

_(The latter of this pair seems significantly harder than the former.)_
     
_And then, of course, the canonical challenge of this sort:_

* To recognize the prime numbers, expressed either in base 2 or in base 10.

_Each of the above challenges involves a number-theoretical category that can easily be described in purely syntactic terms (i.e., as a rule-based pattern of symbols).  It would be trivial to generate millions of examples of such categories mechanically, and then you just feed them to the neural net.  You can also feed the network lots of counterexamples -- marking them, of course, as non-members of the category.  Can a deep neural network learn any of these categories?  All of them?  Some of them?_

---

In this assignment, you will use an RNN to try to solve the divisibility-by-3 problem (the rest are challenges you might want to try in your free time!): 

* **"To recognize the multiples of 3, expressed in base 10."**  Specifically, you must:
    * Design an RNN that takes a sequence of digits as input. 
    * Represent digits in base 10 by using a categorical, one-hot encoding, with one node for each digit from 0 through 9.
    * Train the RNN to categorize a number as True if it is evenly divisible by 3, False otherwise.
    * Test the RNN on a set of previously-unseen numbers, including numbers that are 4 digits long, such as 2225 and 3333.
    * Acheive an accuracy of at least 95% on the test set (report the accuracy in the cell marked below).
    * Answer the questions at the end of this notebook.




- [One hot encoding](https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/) 
- [Sequence Prediction](https://machinelearningmastery.com/sequence-prediction-problems-learning-lstm-recurrent-neural-networks/)
- [LSTM Tutorial Keras](https://adventuresinmachinelearning.com/keras-lstm-tutorial/)

# Setup
## Imports

- [Keras - Guide to the sequential model](https://keras.io/getting-started/sequential-model-guide/)
- [Input](https://keras.io/layers/core/)
- [GRU](https://keras.io/layers/recurrent/)
- [LSTM](https://keras.io/layers/recurrent/)
- [Dense](https://keras.io/layers/core/#Dense)
- [Masking](https://keras.io/layers/core/)
- [Dropout](https://keras.io/layers/core/)

- [Keras - fit](https://keras.io/models/sequential/)
- [Recurrent Neural Networks by Example in Python](https://towardsdatascience.com/recurrent-neural-networks-by-example-in-python-ffd204f99470)
- [Sequence Classification with LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/)

Callback in Keras

- [Machine learning Mastery](https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/) 
- [Early Stopping](https://chrisalbon.com/deep_learning/keras/neural_network_early_stopping/) 
- [Keras - Callback](https://keras.io/callbacks/)

Loading & saving model

- [Loading & Saving](https://jovianlin.io/saving-loading-keras-models/)

## Model:

In [25]:
%matplotlib inline
from keras.models import Model, Sequential, load_model
from keras.layers import Input, GRU, LSTM, Dense, Masking, Dropout, InputLayer, TimeDistributed
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.regularizers import l1_l2
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [26]:
# Set up params for dataset.
DIVISIBILITY_NUMBER = 3         # We want to test for divisilibity by 3.
TRAIN_TEST_SPLIT = 0.8          # Percentage of data in training set
NUM_EXAMPLES_PER_CLASS = 1000   # Generate the first 1000 multiples of 3 for training/testing
                                # Also generate 1000 non-multiples of 3.
NUM_CATEGORIES = 10             # 10 digits
MAX_DIGITS = 5                  # Number of digits allowed in input strings

# Neural net hyperparameters-- just an example. Adjust these as needed.
BATCH_SIZE = 32
NUM_LSTM_NODES = 10             
DROPOUT = 0.5
LEARING_RATE = 0.001
NUM_EPOCHS = 2500

# TODO: add/modify constants as needed

In [27]:
from random import sample

def generate_example_numbers(base_number=DIVISIBILITY_NUMBER, num_examples_per_class=NUM_EXAMPLES_PER_CLASS):
    """Return a tuple of two lists: (list_of_multiples, list_of_nonmultiples).
    
    For example, ([0, 3, 6, 9, 12, ...2997], [1, 4, 5, 8, 11, 13, 14,...,2999]).
    Each list contains num_examples_per_class elements.
    """
 
    multiples = [n * base_number for n in range(num_examples_per_class)]
    non_multiples = [n * (base_number+1) for n in range(num_examples_per_class)]
    return (multiples, non_multiples)

In [28]:
def digit_to_vector(digit):
    """Given a digit from 0-9, return a numpy array representing the digit using a 1-hot encoding.
    keras.utils.to_categorical may be useful.
    """
    tmp = to_categorical(digit, dtype='int',num_classes=NUM_CATEGORIES)
    return tmp

In [29]:
def number_to_input_example(number, max_digits=MAX_DIGITS):
    """Given an integer number, return a numpy float array of 0.0s and 1.0s, of the correct shape to feed into the 
    neural net.
    
    For example, if you have a max of 5 digits then you should have a 2D numpy matrix: 5 rows (one for each
    sequence index), and 10 columns (1 for each digit).
    
    In order to train in "batch" mode, the RNN expects every example to have the same shape. So if you have a 2-digit
    number such as "42", you need to pad the example with a "padding" token somehow; for example, "???42", and then
    use keras.layers.Masking to ignore the leading digits. Or just pad with 0s, as in "00042". 
    keras.preprocessing.sequence.pad_sequences can help with this.
    """
    padded_number = pad_sequences([[int(k) for k in str(number)]], maxlen=max_digits)
    result = [np.zeros(10) for _ in range(max_digits)]
    for i, k in zip(range(max_digits), padded_number[0]):
      result[i][k] = 1
    
    return result

    

In [30]:
number_to_input_example(42, max_digits=MAX_DIGITS)

[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])]

In [31]:
def generate_dataset(divisibility_number=DIVISIBILITY_NUMBER, 
                     train_test_split=TRAIN_TEST_SPLIT, 
                     num_examples_per_class=NUM_EXAMPLES_PER_CLASS):
    """Generate a dataset ready for training. Returns a list of tuples. Each tuple is of the form
    (input_array, label). The dataset should be shuffled either here or during the training process to
    mix divisile-by-DIVISIBILITY_NUMBER and not-divisible-by-DIVISIBILITY_NUMBER examples.
    The dataset should consist of NUM_EXAMPLES_PER_CLASS positive examples (e.g., 1000 examples of divisible-by-3), and
    also NUM_EXAMPLES_PER_CLASS negative examples (e.g., 1000 examples of not-divisible-by-3).
    """
    example_numbers = generate_example_numbers(divisibility_number, num_examples_per_class)
    divisible = example_numbers[0]
    non_divisible = example_numbers[1]
    np.random.shuffle(divisible)
    np.random.shuffle(non_divisible)
    
    result = []
    for k, m in zip(divisible, non_divisible):
        result.append((k, 1))
        result.append((m, 0))
        
    
    return result




In [32]:
# Build RNN model.
def build_model():
    model = Sequential()
    model.add(LSTM(NUM_LSTM_NODES, dropout=DROPOUT, recurrent_dropout=DROPOUT, batch_input_shape=(1,MAX_DIGITS,10), return_sequences=True))  # Use return_sequences=True for multiple hidden layers
    model.add(Dense(NUM_LSTM_NODES))
    model.add(LSTM(NUM_LSTM_NODES, dropout=DROPOUT, recurrent_dropout=DROPOUT, batch_input_shape=(1,MAX_DIGITS,10), return_sequences=False))  # Use return_sequences=True for multiple hidden layers
    model.add(Dense(1))  # Model should return 1 or 0 for divisible/not-divisible
    
    return model

In [33]:
model = build_model()

# Compile model
adam = Adam(lr=LEARING_RATE)   # Modify learning algorithm as needed
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

# Summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, 5, 10)                840       
_________________________________________________________________
dense_1 (Dense)              (1, 5, 10)                110       
_________________________________________________________________
lstm_2 (LSTM)                (1, 10)                   840       
_________________________________________________________________
dense_2 (Dense)              (1, 1)                    11        
Total params: 1,801
Trainable params: 1,801
Non-trainable params: 0
_________________________________________________________________


In [34]:
data = generate_dataset()
X = np.array([number_to_input_example(d[0]) for d in data])
y = [d[1] for d in data]
n = len(X)

train_len = int(TRAIN_TEST_SPLIT * n)
train_inputs = X[:train_len]
train_labels = y[:train_len]
validation_inputs = X[train_len:]
validation_labels = y[train_len:]





In [35]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1500)
mc = ModelCheckpoint('best_model_3.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [36]:
history = model.fit(train_inputs, train_labels, validation_data=(validation_inputs, validation_labels), 
          batch_size=1, epochs=NUM_EPOCHS, verbose=1, callbacks=[es, mc])


Train on 1600 samples, validate on 400 samples
Epoch 1/2500
1600/1600 [==============================] - 23s 14ms/step - loss: 0.8317 - acc: 0.5075 - val_loss: 0.6872 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to best_model_3.h5
Epoch 2/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.6856 - acc: 0.5500 - val_loss: 0.6554 - val_acc: 0.5850

Epoch 00002: val_acc improved from 0.50000 to 0.58500, saving model to best_model_3.h5
Epoch 3/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.6564 - acc: 0.5906 - val_loss: 0.5959 - val_acc: 0.7275

Epoch 00003: val_acc improved from 0.58500 to 0.72750, saving model to best_model_3.h5
Epoch 4/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.6390 - acc: 0.6606 - val_loss: 0.4841 - val_acc: 0.7750

Epoch 00004: val_acc improved from 0.72750 to 0.77500, saving model to best_model_3.h5
Epoch 5/2500
1600/1600 [============================

1600/1600 [==============================] - 20s 13ms/step - loss: 0.4152 - acc: 0.6469 - val_loss: 0.3422 - val_acc: 0.5825

Epoch 00043: val_acc did not improve from 0.78750
Epoch 44/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4214 - acc: 0.6431 - val_loss: 0.3789 - val_acc: 0.6400

Epoch 00044: val_acc did not improve from 0.78750
Epoch 45/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4401 - acc: 0.6700 - val_loss: 0.3488 - val_acc: 0.5875

Epoch 00045: val_acc did not improve from 0.78750
Epoch 46/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4382 - acc: 0.6262 - val_loss: 0.3615 - val_acc: 0.5900

Epoch 00046: val_acc did not improve from 0.78750
Epoch 47/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4317 - acc: 0.6431 - val_loss: 0.3488 - val_acc: 0.5800

Epoch 00047: val_acc did not improve from 0.78750
Epoch 48/2500
1600/1600 [==============================] - 20s 1

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3975 - acc: 0.7462 - val_loss: 0.3164 - val_acc: 0.6825

Epoch 00086: val_acc did not improve from 0.78750
Epoch 87/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3986 - acc: 0.7700 - val_loss: 0.3183 - val_acc: 0.6925

Epoch 00087: val_acc did not improve from 0.78750
Epoch 88/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4104 - acc: 0.7900 - val_loss: 0.3243 - val_acc: 0.8000

Epoch 00088: val_acc improved from 0.78750 to 0.80000, saving model to best_model_3.h5
Epoch 89/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3905 - acc: 0.7744 - val_loss: 0.3199 - val_acc: 0.6625

Epoch 00089: val_acc did not improve from 0.80000
Epoch 90/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4103 - acc: 0.7169 - val_loss: 0.3163 - val_acc: 0.6575

Epoch 00090: val_acc did not improve from 0.80000
Epoch 91/2500
1600/1600 [==

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3703 - acc: 0.8263 - val_loss: 0.3126 - val_acc: 0.8550

Epoch 00128: val_acc did not improve from 0.85500
Epoch 129/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3646 - acc: 0.8331 - val_loss: 0.3119 - val_acc: 0.8550

Epoch 00129: val_acc did not improve from 0.85500
Epoch 130/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4044 - acc: 0.8137 - val_loss: 0.3132 - val_acc: 0.8550

Epoch 00130: val_acc did not improve from 0.85500
Epoch 131/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3734 - acc: 0.8281 - val_loss: 0.3134 - val_acc: 0.8550

Epoch 00131: val_acc did not improve from 0.85500
Epoch 132/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3784 - acc: 0.8263 - val_loss: 0.3087 - val_acc: 0.8550

Epoch 00132: val_acc did not improve from 0.85500
Epoch 133/2500
1600/1600 [==============================] - 

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3904 - acc: 0.8231 - val_loss: 0.3166 - val_acc: 0.8550

Epoch 00171: val_acc did not improve from 0.85500
Epoch 172/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3912 - acc: 0.8319 - val_loss: 0.3083 - val_acc: 0.8550

Epoch 00172: val_acc did not improve from 0.85500
Epoch 173/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3559 - acc: 0.8306 - val_loss: 0.3094 - val_acc: 0.8550

Epoch 00173: val_acc did not improve from 0.85500
Epoch 174/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3718 - acc: 0.8287 - val_loss: 0.3072 - val_acc: 0.8550

Epoch 00174: val_acc did not improve from 0.85500
Epoch 175/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3876 - acc: 0.8256 - val_loss: 0.3097 - val_acc: 0.8550

Epoch 00175: val_acc did not improve from 0.85500
Epoch 176/2500
1600/1600 [==============================] - 

...

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3952 - acc: 0.8300 - val_loss: 0.3069 - val_acc: 0.8550

Epoch 02321: val_acc did not improve from 0.85500
Epoch 2322/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3886 - acc: 0.8344 - val_loss: 0.3064 - val_acc: 0.8550

Epoch 02322: val_acc did not improve from 0.85500
Epoch 2323/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4265 - acc: 0.8219 - val_loss: 0.3064 - val_acc: 0.8550

Epoch 02323: val_acc did not improve from 0.85500
Epoch 2324/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3738 - acc: 0.8375 - val_loss: 0.3067 - val_acc: 0.8550

Epoch 02324: val_acc did not improve from 0.85500
Epoch 2325/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3529 - acc: 0.8438 - val_loss: 0.3063 - val_acc: 0.8550

Epoch 02325: val_acc did not improve from 0.85500
Epoch 2326/2500
1600/1600 [=============================

1600/1600 [==============================] - 20s 13ms/step - loss: 0.4293 - acc: 0.8331 - val_loss: 0.3059 - val_acc: 0.8550

Epoch 02364: val_acc did not improve from 0.85500
Epoch 2365/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.4231 - acc: 0.8319 - val_loss: 0.3059 - val_acc: 0.8550

Epoch 02365: val_acc did not improve from 0.85500
Epoch 2366/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4204 - acc: 0.8044 - val_loss: 0.3082 - val_acc: 0.8550

Epoch 02366: val_acc did not improve from 0.85500
Epoch 2367/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3508 - acc: 0.8394 - val_loss: 0.3058 - val_acc: 0.8550

Epoch 02367: val_acc did not improve from 0.85500
Epoch 2368/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3606 - acc: 0.8369 - val_loss: 0.3056 - val_acc: 0.8550

Epoch 02368: val_acc did not improve from 0.85500
Epoch 2369/2500
1600/1600 [=============================

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3952 - acc: 0.8269 - val_loss: 0.3079 - val_acc: 0.8550

Epoch 02407: val_acc did not improve from 0.85500
Epoch 2408/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3878 - acc: 0.8337 - val_loss: 0.3174 - val_acc: 0.8550

Epoch 02408: val_acc did not improve from 0.85500
Epoch 2409/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3851 - acc: 0.8375 - val_loss: 0.3099 - val_acc: 0.8550

Epoch 02409: val_acc did not improve from 0.85500
Epoch 2410/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3769 - acc: 0.8412 - val_loss: 0.3065 - val_acc: 0.8550

Epoch 02410: val_acc did not improve from 0.85500
Epoch 2411/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3781 - acc: 0.8294 - val_loss: 0.3068 - val_acc: 0.8550

Epoch 02411: val_acc did not improve from 0.85500
Epoch 2412/2500
1600/1600 [=============================

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3994 - acc: 0.8306 - val_loss: 0.3065 - val_acc: 0.8550

Epoch 02450: val_acc did not improve from 0.85500
Epoch 2451/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3715 - acc: 0.8325 - val_loss: 0.3129 - val_acc: 0.8550

Epoch 02451: val_acc did not improve from 0.85500
Epoch 2452/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4134 - acc: 0.8287 - val_loss: 0.3176 - val_acc: 0.8550

Epoch 02452: val_acc did not improve from 0.85500
Epoch 2453/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3911 - acc: 0.8331 - val_loss: 0.3107 - val_acc: 0.8550

Epoch 02453: val_acc did not improve from 0.85500
Epoch 2454/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3831 - acc: 0.8269 - val_loss: 0.3073 - val_acc: 0.8550

Epoch 02454: val_acc did not improve from 0.85500
Epoch 2455/2500
1600/1600 [=============================

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3931 - acc: 0.8337 - val_loss: 0.3086 - val_acc: 0.8550

Epoch 02493: val_acc did not improve from 0.85500
Epoch 2494/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3962 - acc: 0.8369 - val_loss: 0.3055 - val_acc: 0.8550

Epoch 02494: val_acc did not improve from 0.85500
Epoch 2495/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4012 - acc: 0.8325 - val_loss: 0.3073 - val_acc: 0.8550

Epoch 02495: val_acc did not improve from 0.85500
Epoch 2496/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3846 - acc: 0.8381 - val_loss: 0.3101 - val_acc: 0.8550

Epoch 02496: val_acc did not improve from 0.85500
Epoch 2497/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3553 - acc: 0.8400 - val_loss: 0.3074 - val_acc: 0.8550

Epoch 02497: val_acc did not improve from 0.85500
Epoch 2498/2500
1600/1600 [=============================

In [37]:
model.save("Guillaume_model_June_3.h5")

In [38]:
scores = model.evaluate(validation_inputs, validation_labels, batch_size=1,verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.50%


In [39]:
%pwd

'C:\\Users\\guill\\Desktop\\UW_ML_Q3_Deep_Learning\\Lab_5'

In [40]:
model = load_model("Guillaume_model_June.h5")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (1, 5, 10)                840       
_________________________________________________________________
dense_7 (Dense)              (1, 5, 10)                110       
_________________________________________________________________
lstm_8 (LSTM)                (1, 10)                   840       
_________________________________________________________________
dense_8 (Dense)              (1, 1)                    11        
Total params: 1,801
Trainable params: 1,801
Non-trainable params: 0
_________________________________________________________________


## Report your final accuracy on the validation dataset below.

In [41]:
print('accuracy is {}'.format(round(scores[1]*100,2)))

accuracy is 85.5


## Examine model outputs

# Further Questions

1) What happens if you give a 5-digit number or a 6-digit number to the trained model, after training on 1-, 2-, 3-, and 4- digit numbers?

In [42]:
ex = np.array([number_to_input_example(420000, max_digits=6)])
ex

array([[[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [43]:
try:
    model.predict(ex)
except ValueError:
    print('ValueError')

ValueError


2) Pick another number from Hofstadter's list above, such as 9, 7, or 29. Train a model, and report the accuracy of your results. Did it work or not? Why or why not (your best guess)?

In [44]:
# Set up params for dataset.
DIVISIBILITY_NUMBER = 9         # We want to test for divisilibity by 3.
TRAIN_TEST_SPLIT = 0.8          # Percentage of data in training set
NUM_EXAMPLES_PER_CLASS = 1000   # Generate the first 1000 multiples of 3 for training/testing
                                # Also generate 1000 non-multiples of 3.
NUM_CATEGORIES = 10             # 10 digits
MAX_DIGITS = 5                  # Number of digits allowed in input strings

# Neural net hyperparameters-- just an example. Adjust these as needed.
BATCH_SIZE = 32
NUM_LSTM_NODES = 10             
DROPOUT = 0.5
LEARING_RATE = 0.001
NUM_EPOCHS = 2500

# TODO: add/modify constants as needed

In [45]:
# Print the model configuration.
model = build_model()

# Compile model
adam = Adam(lr=LEARING_RATE)   # Modify learning algorithm as needed
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

# Summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (1, 5, 10)                840       
_________________________________________________________________
dense_3 (Dense)              (1, 5, 10)                110       
_________________________________________________________________
lstm_4 (LSTM)                (1, 10)                   840       
_________________________________________________________________
dense_4 (Dense)              (1, 1)                    11        
Total params: 1,801
Trainable params: 1,801
Non-trainable params: 0
_________________________________________________________________


In [46]:
data = generate_dataset()
X = np.array([number_to_input_example(d[0]) for d in data])
y = [d[1] for d in data]
n = len(X)

train_len = int(TRAIN_TEST_SPLIT * n)
train_inputs = X[:train_len]
train_labels = y[:train_len]
validation_inputs = X[train_len:]
validation_labels = y[train_len:]

In [47]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1500)
mc = ModelCheckpoint('best_model_2_9.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
history = model.fit(train_inputs, train_labels, validation_data=(validation_inputs, validation_labels), 
          batch_size=1, epochs=NUM_EPOCHS, verbose=1, callbacks=[es, mc])

model.save("Guillaume_model_June_9.h5")

Train on 1600 samples, validate on 400 samples
Epoch 1/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.7642 - acc: 0.5131 - val_loss: 0.6845 - val_acc: 0.5300

Epoch 00001: val_acc improved from -inf to 0.53000, saving model to best_model_2_9.h5
Epoch 2/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.6955 - acc: 0.5287 - val_loss: 0.6582 - val_acc: 0.6175

Epoch 00002: val_acc improved from 0.53000 to 0.61750, saving model to best_model_2_9.h5
Epoch 3/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.6668 - acc: 0.5812 - val_loss: 0.6158 - val_acc: 0.6850

Epoch 00003: val_acc improved from 0.61750 to 0.68500, saving model to best_model_2_9.h5
Epoch 4/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.6215 - acc: 0.6631 - val_loss: 0.5246 - val_acc: 0.7350

Epoch 00004: val_acc improved from 0.68500 to 0.73500, saving model to best_model_2_9.h5
Epoch 5/2500
1600/1600 [====================

1600/1600 [==============================] - 20s 13ms/step - loss: 0.4187 - acc: 0.6450 - val_loss: 0.3691 - val_acc: 0.6325

Epoch 00043: val_acc did not improve from 0.75500
Epoch 44/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3998 - acc: 0.6469 - val_loss: 0.3736 - val_acc: 0.5975

Epoch 00044: val_acc did not improve from 0.75500
Epoch 45/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4200 - acc: 0.6950 - val_loss: 0.3670 - val_acc: 0.6225

Epoch 00045: val_acc did not improve from 0.75500
Epoch 46/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4375 - acc: 0.6544 - val_loss: 0.3648 - val_acc: 0.6300

Epoch 00046: val_acc did not improve from 0.75500
Epoch 47/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4197 - acc: 0.6725 - val_loss: 0.3623 - val_acc: 0.6275

Epoch 00047: val_acc did not improve from 0.75500
Epoch 48/2500
1600/1600 [==============================] - 20s 1

1600/1600 [==============================] - 20s 13ms/step - loss: 0.4228 - acc: 0.7037 - val_loss: 0.3308 - val_acc: 0.6525

Epoch 00086: val_acc did not improve from 0.76250
Epoch 87/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3937 - acc: 0.7144 - val_loss: 0.3391 - val_acc: 0.6425

Epoch 00087: val_acc did not improve from 0.76250
Epoch 88/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4021 - acc: 0.7000 - val_loss: 0.3427 - val_acc: 0.6925

Epoch 00088: val_acc did not improve from 0.76250
Epoch 89/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3943 - acc: 0.7362 - val_loss: 0.3333 - val_acc: 0.6550

Epoch 00089: val_acc did not improve from 0.76250
Epoch 90/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3614 - acc: 0.7381 - val_loss: 0.3373 - val_acc: 0.6775

Epoch 00090: val_acc did not improve from 0.76250
Epoch 91/2500
1600/1600 [==============================] - 20s 1

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3744 - acc: 0.7581 - val_loss: 0.3283 - val_acc: 0.7425

Epoch 00129: val_acc did not improve from 0.76250
Epoch 130/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3642 - acc: 0.7556 - val_loss: 0.3270 - val_acc: 0.7100

Epoch 00130: val_acc did not improve from 0.76250
Epoch 131/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3723 - acc: 0.7413 - val_loss: 0.3269 - val_acc: 0.6675

Epoch 00131: val_acc did not improve from 0.76250
Epoch 132/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3512 - acc: 0.7538 - val_loss: 0.3356 - val_acc: 0.6875

Epoch 00132: val_acc did not improve from 0.76250
Epoch 133/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3729 - acc: 0.7306 - val_loss: 0.3234 - val_acc: 0.6600

Epoch 00133: val_acc did not improve from 0.76250
Epoch 134/2500
1600/1600 [==============================] - 

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3912 - acc: 0.6525 - val_loss: 0.3263 - val_acc: 0.6300

Epoch 00172: val_acc did not improve from 0.76250
Epoch 173/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3852 - acc: 0.6350 - val_loss: 0.3249 - val_acc: 0.6300

Epoch 00173: val_acc did not improve from 0.76250
Epoch 174/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3755 - acc: 0.6362 - val_loss: 0.3261 - val_acc: 0.6300

Epoch 00174: val_acc did not improve from 0.76250
Epoch 175/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3572 - acc: 0.6581 - val_loss: 0.3245 - val_acc: 0.6375

Epoch 00175: val_acc did not improve from 0.76250
Epoch 176/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3499 - acc: 0.6887 - val_loss: 0.3217 - val_acc: 0.6400

Epoch 00176: val_acc did not improve from 0.76250
Epoch 177/2500
1600/1600 [==============================] - 

...

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3620 - acc: 0.6850 - val_loss: 0.3227 - val_acc: 0.6300

Epoch 00344: val_acc did not improve from 0.76250
Epoch 345/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.4339 - acc: 0.6756 - val_loss: 0.3183 - val_acc: 0.6175

Epoch 00345: val_acc did not improve from 0.76250
Epoch 346/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3764 - acc: 0.6750 - val_loss: 0.3169 - val_acc: 0.6300

Epoch 00346: val_acc did not improve from 0.76250
Epoch 347/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3411 - acc: 0.6731 - val_loss: 0.3192 - val_acc: 0.6300

Epoch 00347: val_acc did not improve from 0.76250
Epoch 348/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3488 - acc: 0.6900 - val_loss: 0.3185 - val_acc: 0.6375

Epoch 00348: val_acc did not improve from 0.76250
Epoch 349/2500
1600/1600 [==============================] - 

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3311 - acc: 0.6381 - val_loss: 0.3173 - val_acc: 0.5625

Epoch 00387: val_acc did not improve from 0.76250
Epoch 388/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3562 - acc: 0.6494 - val_loss: 0.3180 - val_acc: 0.5775

Epoch 00388: val_acc did not improve from 0.76250
Epoch 389/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.3674 - acc: 0.6450 - val_loss: 0.3200 - val_acc: 0.5625

Epoch 00389: val_acc did not improve from 0.76250
Epoch 390/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3866 - acc: 0.6419 - val_loss: 0.3160 - val_acc: 0.6300

Epoch 00390: val_acc did not improve from 0.76250
Epoch 391/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3524 - acc: 0.6300 - val_loss: 0.3174 - val_acc: 0.6250

Epoch 00391: val_acc did not improve from 0.76250
Epoch 392/2500
1600/1600 [==============================] - 

1600/1600 [==============================] - 20s 13ms/step - loss: 0.3274 - acc: 0.6544 - val_loss: 0.3194 - val_acc: 0.5525

Epoch 00430: val_acc did not improve from 0.76250
Epoch 431/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3646 - acc: 0.6531 - val_loss: 0.3173 - val_acc: 0.6075

Epoch 00431: val_acc did not improve from 0.76250
Epoch 432/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.3428 - acc: 0.6537 - val_loss: 0.3224 - val_acc: 0.6300

Epoch 00432: val_acc did not improve from 0.76250
Epoch 433/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3586 - acc: 0.6663 - val_loss: 0.3191 - val_acc: 0.6300

Epoch 00433: val_acc did not improve from 0.76250
Epoch 434/2500
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3742 - acc: 0.6650 - val_loss: 0.3158 - val_acc: 0.6300

Epoch 00434: val_acc did not improve from 0.76250
Epoch 435/2500
1600/1600 [==============================] - 

1600/1600 [==============================] - 21s 13ms/step - loss: 0.3401 - acc: 0.6437 - val_loss: 0.3155 - val_acc: 0.6300

Epoch 00473: val_acc did not improve from 0.76250
Epoch 474/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.3471 - acc: 0.6519 - val_loss: 0.3202 - val_acc: 0.6300

Epoch 00474: val_acc did not improve from 0.76250
Epoch 475/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.3400 - acc: 0.6456 - val_loss: 0.3193 - val_acc: 0.6300

Epoch 00475: val_acc did not improve from 0.76250
Epoch 476/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.3455 - acc: 0.6475 - val_loss: 0.3159 - val_acc: 0.6300

Epoch 00476: val_acc did not improve from 0.76250
Epoch 477/2500
1600/1600 [==============================] - 21s 13ms/step - loss: 0.3699 - acc: 0.6488 - val_loss: 0.3233 - val_acc: 0.6300

Epoch 00477: val_acc did not improve from 0.76250
Epoch 478/2500
1600/1600 [==============================] - 

KeyboardInterrupt: 

In [48]:
#model = load_model("Guillaume_model_June_9.h5")
model = load_model('best_model_2_9.h5')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (1, 5, 10)                840       
_________________________________________________________________
dense_3 (Dense)              (1, 5, 10)                110       
_________________________________________________________________
lstm_4 (LSTM)                (1, 10)                   840       
_________________________________________________________________
dense_4 (Dense)              (1, 1)                    11        
Total params: 1,801
Trainable params: 1,801
Non-trainable params: 0
_________________________________________________________________


In [49]:
scores = model.evaluate(validation_inputs, validation_labels, batch_size=1,verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 76.25%


it does work but I cannot get the same level of accurracy. 


3) Record any other comments/insights from your model training process. What worked well? What caused trouble?

The run duration is huge 

4) If you didn't have a training algorithm, how would you design a RNN-style system to recognize divisibility by 3?
Ignoring the details of the weights, what kind of state must be carried over from step to step as each digit is read in
successively?

I would have read the number backward and carry the divisility  of the digit as a state

5) BONUS (hard): Explain how the neural net you trained above works, with evidence from examining the node activations as the net runs. Does it do anything similar to what you would have designed as a human?